In [ ]:
import py_entitymatching as em
import pandas as pd
import os
from sdv.tabular import CTGAN
from utils.utils import *
from IPython.display import display, HTML

pd.options.mode.chained_assignment = None  # default='warn''

<h3>0. Prelude.</h3>
This pipeline takes care of the CTGAN generation and sampling. <br>
First we decide on either generation just one type or both. <br>
Thereafter we ensure distances between the generated matches attributes. <br>
Lastly we save the data corresponding the wished format. i.e. either a dataframe or as a text. <br>
<br>

In [ ]:
model_dir = r'C:\Users\aleks\Desktop\Master Thesis\Idun\CTGAN\testing_param\textual_time\cpu'
model_A_name = "textual_m_1000_3000.pkl"
model_A_sample_size = 1000

# Determine if we are generating both types of data.
# NOTE: If we are, model_A should always be "matched" !!!
generate_both = False
if generate_both:
    model_B_name = "Placeholder2.pkl"
    model_B_sample_size = 1000
else:
    # Determine what type of singular data we are generating
    generate_matches = True

# If the syntethic dataset should be saved as a dataframe or text 
ditto_format = False 

# The directory for where the dataset will be saved, and its name. 
datasets_dir = r'C:\Users\aleks\Desktop\Master Thesis\Py_Magellan\DataSets\temp'
name_of_sample_set = "generated_set"

# Takes care of file format...
if ditto_format:
    name_of_sample_set = name_of_sample_set + ".txt"
else:
    name_of_sample_set = name_of_sample_set + ".csv"


<h3>1. Load models.</h3>
First, load the CTGAN model(s) from our model path. <br>

In [ ]:
model_A_path = model_dir + os.sep + model_A_name

if generate_both:
    model_B_path = model_dir + os.sep + model_B_name
    model_B = CTGAN.load(model_B_path)

    model_A = CTGAN.load(model_A_path)
else:
    model_A = CTGAN.load(model_A_path)

<h3>2. Generate Matches and/or Non-Matches.</h3>
After loading our model(s), we generate the synthethic sample data. <br>
Then, we ensure that the generated matches follows some coherency between the attributes <br>
between l_table and r_table. We use Levhenstein distance to ensure similarities between the attributes. <br>
Lastly, we concat the tables together if we are generating both types. 

In [ ]:
if generate_both:
    model_A_data = model_A.sample(model_A_sample_size)
    model_B_data = model_B.sample(model_B_sample_size)

    model_A_data = ensure_data(model_A_data, True)
    model_B_data = ensure_data(model_B_data, False)
    data = model_A_data.append(model_B_data, ignore_index=True)
else:
    model_A_data = model_A.sample(100000)
    if generate_matches:
        data = ensure_data(model_A_data, True, 0.6)
    else:
        data = ensure_data(model_A_data, False)

# temp_path = r'C:\Users\aleks\Desktop\Master Thesis\Idun\CTGAN\testing_param\textual_time\cpu\train.txt'

# with open(temp_path, 'r', encoding='utf-8') as file:
#         data_origin = file.read()

# table_A, table_B, truth_table = ditto_reformater(data_origin)

# table_A = table_A.add_prefix("ltable_")
# table_B = table_B.add_prefix("rtable_")
# table = pd.concat([table_A, table_B, truth_table], axis=1)

# table = table[table['Truth'] > 0]
# orig_columns = table.columns.to_list()

# display(HTML(table.drop_duplicates().to_html()))

data = data.drop_duplicates()
# data = data[orig_columns]
display(HTML(data.to_html()))
len(data.index.values)

<h3>3. Lastly, we ensure that the data format is satisfied.</h3>
If we are generating data to be tested for "Magellan", we can conclude and <br>
save the dataframe to a .csv file. <br> <br>
If the data is to be fed to "DITTO", we need to reformat the dataframe to <br>
a text-based format. 

In [ ]:
if ditto_format:
    ditto_data = magellan_reformater(data)

    data_save_path = datasets_dir + os.sep + name_of_sample_set

    with open(data_save_path, "w", encoding="utf-8") as text_file:
        text_file.write(ditto_data)
else:
    data.to_csv(datasets_dir + os.sep + name_of_sample_set)